In [40]:
import pandas as pd
import nltk as nltk
import sklearn
from sklearn import *
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor
import tensorflow as tf
import keras
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer,SentimentIntensityAnalyzer
from nltk.sentiment.util import *
from nltk import tokenize
import numpy as np
import html
import re
pd.set_option(
    'display.max_colwidth', 0
)
np.random.seed(42)

In [162]:
df = pd.read_csv('hacker_news_sample.csv',nrows=100000)


In [166]:
def preprocess_df(df):
    df = df[df['text'].notna()]
    df['neg'], df['neu'], df['pos'], df["compound"] = [np.nan, np.nan,np.nan,np.nan]    
    return df

def add_sentiment(df):
    df['neg'], df['neu'], df['pos'], df["compound"] = [np.nan, np.nan,np.nan,np.nan]
    sid = SentimentIntensityAnalyzer()
    for i, row in df.iterrows():
        text = row["text"]
        text = html.unescape(text)
        text =  re.sub('<[^<]+?>', '', text)
        df.at[i,"text"] = text
        ss = sid.polarity_scores(text)
        for k in ss:
            df.at[i,k] = ss[k]






In [164]:
df = preprocess_df(df)
add_sentiment(df)
df.describe()

,score,time,id,parent,descendants,ranking,neg,neu,pos,compound
count,1737.000000,8.153000e+04,8.153000e+04,7.979300e+04,1548.000000,0.0,81530.000000,81530.000000,81530.000000,81530.000000
mean,5.213587,1.387459e+09,7.369586e+06,7.419128e+06,3.410853,NaN,0.059936,0.825668,0.114299,0.216708
std,21.129398,7.622413e+07,4.220612e+06,4.213057e+06,14.535619,NaN,0.082589,0.134709,0.119930,0.529816
min,0.000000,1.172086e+09,3.620000e+02,1.450000e+02,-1.000000,NaN,0.000000,0.000000,0.000000,-0.999500
25%,1.000000,1.332827e+09,3.760196e+06,3.836577e+06,-1.000000,NaN,0.000000,0.761000,0.018000,-0.077200
50%,1.000000,1.394251e+09,7.364090e+06,7.439650e+06,0.000000,NaN,0.034000,0.839000,0.094000,0.273200
75%,3.000000,1.454520e+09,1.102772e+07,1.106604e+07,2.000000,NaN,0.092000,0.912000,0.161000,0.680800
max,639.000000,1.498466e+09,1.463506e+07,1.463497e+07,368.000000,NaN,1.000000,1.000000,1.000000,0.999700


In [56]:
X_train, X_test = train_test_split(df)


In [119]:
scored = X_train.loc[X_train['compound'] != 0]

In [128]:
X = pipeline.fit_transform(scored["text"])

## Decision Tree

In [ ]:
pipeline = Pipeline(
    [
        ('vect', CountVectorizer(analyzer='word')),
        ('tfidf', TfidfTransformer()),    
    ]
)

In [122]:
tree_reg = DecisionTreeRegressor(random_state=42)
tree_reg.fit(X, scored["compound"])

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=42, splitter='best')

In [129]:
scored_test = X_test.loc[X_test['compound'] != 0]
X2 = pipeline.transform(scored_test["text"])
print(scored_test["compound"].shape)
print(X2.shape)

In [133]:
from sklearn.metrics import mean_squared_error
predictions = tree_reg.predict(X2)

tree_mse = mean_squared_error(scored_test["compound"], predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

0.6365505601380854

## Keras 

In [152]:

EPOCHS = 10

def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[X.shape[1]]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

In [153]:
model = build_model()
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 64)                4916992   
_________________________________________________________________
dense_4 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 65        
Total params: 4,921,217
Trainable params: 4,921,217
Non-trainable params: 0
_________________________________________________________________


In [156]:
history = model.fit(
  X, scored["compound"],
  epochs=EPOCHS, validation_split = 0.2, verbose=1)

Train on 41576 samples, validate on 10395 samples
Epoch 1/10
41576/41576 [==============================] - 102s 2ms/sample - loss: 0.1676 - mae: 0.3143 - mse: 0.1676 - val_loss: 0.1464 - val_mae: 0.2798 - val_mse: 0.1464
Epoch 2/10
41576/41576 [==============================] - 110s 3ms/sample - loss: 0.1047 - mae: 0.2321 - mse: 0.1047 - val_loss: 0.1346 - val_mae: 0.2717 - val_mse: 0.1346
Epoch 3/10
41576/41576 [==============================] - 105s 3ms/sample - loss: 0.0793 - mae: 0.1950 - mse: 0.0793 - val_loss: 0.1419 - val_mae: 0.2650 - val_mse: 0.1419
Epoch 4/10
41576/41576 [==============================] - 111s 3ms/sample - loss: 0.0593 - mae: 0.1650 - mse: 0.0593 - val_loss: 0.1399 - val_mae: 0.2785 - val_mse: 0.1399
Epoch 5/10
41576/41576 [==============================] - 105s 3ms/sample - loss: 0.0442 - mae: 0.1402 - mse: 0.0442 - val_loss: 0.1430 - val_mae: 0.2678 - val_mse: 0.1430
Epoch 6/10
41576/41576 [==============================] - 105s 3ms/sample - loss: 0.0335 -

In [157]:
def predict_text(text, model = model):
    xtemp = pipeline.transform([text])
    predictions = model.predict(xtemp)
    print(predictions)
    return predictions



txt = "Reality is that while China blocks Facebook, Google, etc and smartly props up their own clones, it’s “aghast” at the American protectionism and xenophobic behaviour. How dare the Americans block a Chinese app?! China is not a democracy. It’s not interested in fairness. China is playing the long game. Just like the wars of the past were fought with little toy armies of a few thousand knights and noblemen marching into each other’s countries until someone decided to conscript their whole nation into battle, the West is fighting allowing China to pilfer its technology, wreak the environment, and compete with state backed organisations. Wanna compete with Huawei? Good luck sending in your company noblemen, China is sending their whole nation behind it."


preds = predict_text("Impressive! Personally, I am a few thousand places behind, but still in the top 0.2%. How? I asked and answered a few hundred questions early on, years ago, when SO was new and interesting... Now those answers are old and, like most answers on SO, out of date. Usually when I google something technical and get seemingly the exact right question asked on SO, the answers are no longer correct. Software versions change. And yet I still get a steady trickle of votes, forever increasing the gulf between outdated and no-longer-participating people like me and anyone starting on SO today.")


[[0.6620629]]


In [160]:
predictions = model.predict(X2)


In [161]:
tree_mse = mean_squared_error(scored_test["compound"], predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

0.39776707368686554